In [1]:
import pickle
import numpy as np
import random
import math

from lstm_class import RNNLM
from bilstm_class import RNNLM_bilstm
from rnn_class import RNNModel
from model_functions import model_factory

models=['bigram','trigram','rnn','lstm','bilstm','bert','bert_whole_word','roberta','xlm','electra','gpt2']

#load probability ranges and get steps
rngs=np.load('prob_ranges_10_90.npy')
steps_all=np.zeros([11,10])
for i,rng in enumerate(rngs):
    low=rng[0]-65
    high=rng[1]
    steps=np.linspace(low,high,10)
    steps_all[i,:]=steps

#turn on/off printing (1=on)
print_on=1

#model names
model1_name='lstm'

#bigram and trigram models run on CPU, so gpu_id will be ignored
model1_gpu_id=0

#get probability range for model 1
steps=steps_all[models.index(model1_name)]

#sentence length
sent_len=8

In [2]:
model1=model_factory(model1_name,model1_gpu_id)

get_model1_sent_prob=model1.sent_prob
get_model1_word_probs=model1.word_probs

/home/columbiadpml/anaconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/columbiadpml/anaconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/columbiadpml/anaconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by ac

In [3]:
from vocabulary import vocab_low, vocab_low_freqs, vocab_cap, vocab_cap_freqs

In [4]:
for step_ind in range(8,10):
    
    step=steps[step_ind]
    
    file=open(model1_name+'_level_'+str(step_ind+1)+'.txt','w')
    
    num_sents=0
    while num_sents<60:
    
        wordi=np.arange(sent_len)
        wordis=[]  
        for i in range(1000):
            random.shuffle(wordi)
            wordis=wordis+list(wordi)

        vocab_low_freqs1=np.ones([len(vocab_low_freqs)])/len(vocab_low_freqs)
        vocab_cap_freqs1=np.ones([len(vocab_cap_freqs)])/len(vocab_cap_freqs)

        words1=list(np.random.choice(vocab_cap, 1, p=vocab_cap_freqs1)) + list(np.random.choice(vocab_low, sent_len-1, p=vocab_low_freqs1, replace=False))
                        
        words1o=words1.copy()

        sent1=' '.join(words1)
        
        sent1_last=sent1

        model1_sent1_prob=get_model1_sent_prob(sent1)

        if print_on==1:
            print('Target: '+str(step))
            print('Current: '+str(model1_sent1_prob))
            print(sent1)
            print('\n')

        probs_all_list=[model1_sent1_prob]

        cycle=0
        
   
        for samp in range(10000):       
        
        
        
            if np.abs(model1_sent1_prob-step) < 1:

                file.write(sent1+'.')
                file.write('\n')

                num_sents+=1

                break   
                
            elif cycle==sent_len:
                break

            elif model1_sent1_prob - step > 1:
                break

                
                
                
            if samp%sent_len==0:
                cycle=0

            words1o=words1.copy()

            wordi=int(wordis[samp])

            cur_word1=words1[wordi]

            if wordi==0:
                vocab=vocab_cap
            else:
                vocab=vocab_low

            model1_word1_probs = get_model1_word_probs(words1,wordi)


            if len(model1_word1_probs)==2:
                model1_word1_inds=model1_word1_probs[1]
                model1_word1_probs=model1_word1_probs[0]
            else:
                model1_word1_inds=np.arange(len(vocab))

            words1=words1o.copy()

            word1_list=[vocab[w] for w in model1_word1_inds]

#             model1_word1_probs=model1_word1_probs/np.sum(model1_word1_probs)
        
            word1_tops=[word1_list[vp] for vp in np.argsort(model1_word1_probs)[::-1][:50]] + [cur_word1]
            
     
 

            model1_sent1_probs=[]
            model1_sent1_prob_diffs=[]

            sent1_conts12=[]
            sent2_conts21=[]

            for word1 in word1_tops:

                words1t=words1o.copy()

                words1t[wordi]=word1

                sent1t=' '.join(words1t)
                
                model1_sent1t_prob=get_model1_sent_prob(sent1t)

                model1_sent1_probs.append(model1_sent1t_prob)
                
                if model1_sent1t_prob - step <= 1:
                    break
                
                #model1_sent1_prob_diffs.append(np.abs(model1_sent1t_prob-step))
                
              
            #aa=np.argmin(model1_sent1_prob_diffs)          
            #new_word1=word1_tops[aa]
            
            new_word1=word1

            new_word1o=new_word1

            words1[wordi]=new_word1.upper()

            sent1p=' '.join(words1)

            words1[wordi]=new_word1.lower()
            if wordi==0 or new_word1o[0].isupper():
                words1[wordi]=new_word1.lower().capitalize()

            sent1=' '.join(words1)
            
            if sent1==sent1_last:
                cycle+=1

            sent1_last=sent1

            model1_sent1_prob=model1_sent1t_prob

            probs_all_list.append(model1_sent1_prob)

            if print_on==1:
                print('Target: '+str(step))
                print('Current: '+str(model1_sent1_prob))
                print(sent1p)
                print('\n')

            
            
                
            



Target: -41.09835778454662
Current: -106.11363220214844
Armored laps caught off lightly Venice airstrike hazardous


Target: -41.09835778454662
Current: -85.51761627197266
Armored laps caught off lightly Venice WAS hazardous


Target: -41.09835778454662
Current: -81.98390197753906
Armored laps caught IN lightly Venice was hazardous


Target: -41.09835778454662
Current: -72.77503967285156
Armored TRANSPORT caught in lightly Venice was hazardous


Target: -41.09835778454662
Current: -68.90215301513672
Armored transport TAKEN in lightly Venice was hazardous


Target: -41.09835778454662
Current: -63.512176513671875
Armored transport taken in lightly Venice was UNCLEAR


Target: -41.09835778454662
Current: -53.924381256103516
Armored transport taken in lightly DAMAGE was unclear


Target: -41.09835778454662
Current: -45.841949462890625
THE transport taken in lightly damage was unclear


Target: -41.09835778454662
Current: -42.7121467590332
The transport taken in TO damage was unclear


Targ

KeyboardInterrupt: 

In [ ]:
rngs